### 제주도 도로 교통량 예측 AI 경진대회

#### CatBoost + XGBoost + LGBM Ensemble 모델 (Private 3.08359) (게더타운주민들)

In [4]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import gc

from sklearn.preprocessing import LabelEncoder
from haversine import haversine
from sklearn.cluster import KMeans

import math

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

### 1. csv to parquet

In [6]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [ ]:
csv_to_parquet('./train.csv', 'train')
csv_to_parquet('./test.csv', 'test')

In [ ]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')